In [ ]:
import re
import nltk
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')

from sklearn.feature_selection.univariate_selection import SelectKBest, chi2, f_classif
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import SGDClassifier, SGDRegressor,LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import random
import itertools

import sys
import os
import argparse
from sklearn.pipeline import Pipeline
from scipy.sparse import csr_matrix
from sklearn.feature_extraction.text import CountVectorizer
import six
from abc import ABCMeta
from scipy import sparse
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils import check_X_y, check_array
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.svm import LinearSVC

from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer
from collections import defaultdict
from keras.layers.convolutional import Convolution1D
from keras import backend as K
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
plt.style.use('ggplot')

# Logic Regression 1

In [ ]:
data = pd.read_csv('/Users/wuzijing/Desktop/HKUST/MFIT5004/热点新闻标题预测股票指数趋势/Combined_News_DJIA.csv')
train, test = data[:1691], data[1691:]

In [ ]:
trainheadlines = []

for row in range(0,len(train.index)):
    trainheadlines.append(' '.join(str(x) for x in train.iloc[row,2:27]))

In [ ]:
trainheadlines

In [ ]:
basicvectorizer = CountVectorizer()
basictrain = basicvectorizer.fit_transform(trainheadlines)
print(basictrain.shape)

In [ ]:
print basictrain

In [19]:
basicmodel = LogisticRegression()
basicmodel = basicmodel.fit(basictrain, train["Label"])

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [20]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
basictest = basicvectorizer.transform(testheadlines)
preds1 = basicmodel.predict(basictest)
acc1=accuracy_score(test['Label'], preds1)


In [21]:
print('Logic Regression 1 accuracy: ',acc1 )

Logic Regression 1 accuracy:  0.4441340782122905


In [22]:
basicwords = basicvectorizer.get_feature_names()
basiccoeffs = basicmodel.coef_.tolist()[0]
coeffdf = pd.DataFrame({'Word' : basicwords, 
                        'Coefficient' : basiccoeffs})
coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
coeffdf.head(5)

,Word,Coefficient
19523,nigeria,0.460504
29437,tv,0.448181
25390,self,0.441919
16081,korea,0.416437
5130,canadian,0.414911


In [23]:
coeffdf.tail(5)

,Word,Coefficient
17040,low,-0.471855
27443,students,-0.477279
12883,hacking,-0.479318
24882,sanctions,-0.580962
24667,run,-0.617878


In [24]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix

print (classification_report(test["Label"], preds1))
print (accuracy_score(test["Label"], preds1))

              precision    recall  f1-score   support

           0       0.42      0.38      0.40       175
           1       0.46      0.51      0.48       183

    accuracy                           0.44       358
   macro avg       0.44      0.44      0.44       358
weighted avg       0.44      0.44      0.44       358

0.4441340782122905


In [25]:
pd.crosstab(test["Label"], preds1, rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,66,109
1,90,93


# Logic Regression2

In [26]:
advancedvectorizer = TfidfVectorizer( min_df=0.03, max_df=0.97, max_features = 200000, ngram_range = (2, 2))
advancedtrain = advancedvectorizer.fit_transform(trainheadlines)

In [27]:
print(advancedtrain.shape)                          

(1631, 668)


In [28]:
advancedmodel = LogisticRegression()
advancedmodel = advancedmodel.fit(advancedtrain, train["Label"])

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [29]:
testheadlines = []
for row in range(0,len(test.index)):
    testheadlines.append(' '.join(str(x) for x in test.iloc[row,2:27]))
advancedtest = advancedvectorizer.transform(testheadlines)
preds2 = advancedmodel.predict(advancedtest)
acc2=accuracy_score(test['Label'], preds2)

In [30]:
print('Logic Regression 2 accuracy: ', acc2)


Logic Regression 2 accuracy:  0.5614525139664804


In [31]:
advwords = advancedvectorizer.get_feature_names()
advcoeffs = advancedmodel.coef_.tolist()[0]
advcoeffdf = pd.DataFrame({'Words' : advwords, 
                        'Coefficient' : advcoeffs})
advcoeffdf = advcoeffdf.sort_values(['Coefficient', 'Words'], ascending=[0, 1])
advcoeffdf.head(5)

,Words,Coefficient
396,right to,1.241642
14,after the,1.207601
31,and other,1.201830
281,likely to,1.161108
417,set to,1.160610


In [32]:
advcoeffdf.tail(5)

,Words,Coefficient
1,10 000,-1.066879
371,phone hacking,-1.137198
608,up in,-1.148046
40,around the,-1.153018
459,the country,-1.368953


In [33]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix

print (classification_report(test["Label"], preds2))
print (accuracy_score(test["Label"], preds2))

              precision    recall  f1-score   support

           0       0.57      0.39      0.47       175
           1       0.55      0.72      0.63       183

    accuracy                           0.56       358
   macro avg       0.56      0.56      0.55       358
weighted avg       0.56      0.56      0.55       358

0.5614525139664804
